# Hybrid Flow PoC
It is called hybrid flow because it mixes OAuth2 Authorization Code Flow with OpenID Connect (OIDC).  
This flow allows a page to render without blocking on authorization code redemption to complete.

### Hybrid flow piggy backs on the Auth Code Flow, with required additions/updates in 3 parameters and additional restrictions on the `response_mode` parameter:   
New `scopes`:
- Must include ID Token scopes: `openid` is required, while `profile` and `email` are optional.

New `response_type`: The **`response_type`** must be a combination of at least 2 or all 3 of: **code**, **id_token** and **token**
- **`id_token`** specifically refers to Authentication using **OpenID Connect (OIDC)**
- Possible combinations of `response_type` as defined per [OpenID Connect spec](https://openid.net/specs/oauth-v2-multiple-response-types-1_0.html#Combinations)
    - `code` `token`
    - `code` `id_token`
    - `id_token` `token`
    - `code` `id_token` `token`
> [!NOTE] I would like to highlight a nuance here: when the `id_token` + `code` are requested together, it is what is define as the actual **Hybrid Flow**. Although, as mentioned you can use any combination listed.

New `nonce`: A randomly generated value that the app can verify to mitigate token replay attacks.   

`response_mode`:
- The default when using Hybrid Flow will be `fragment`
- `form_post` is recommended by MSFT for apps, especially when using http://localhost as a redirect URI.    
- Must not be `query` and it is not allowed per the spec     
     
---     
> #### Hybrid Flow requires user interaction. The user must sign-in interactively and satisfy the MFA policy.
To visualize it looks like so:

```sql
Client ----> [Authorization Endpoint]
                       \
                        User Authenticates
                       /
Client <---- [Authorization Code + Tokens]
           |
           | (POST with code)
           v
[Token Endpoint] ----> [Additional Tokens]

```

---     

In [ ]:
import sys
sys.path.append('../')
import OAuth2_Flows
import pyperclip

## Required Variables Setup
Below we are setting up our variables.
- Note that the `redirect_uri` needs to match that on the App Registration

> Note that in the scope you could also request something like: `scope = 'openid email profile offline_access https://graph.microsoft.com/.default'`, and that would also return an id token along with a refresh token. This by the specification is no longer considered Auth Code flow, and it is using the Hybrid Flow leveraging the OpenID Connect (OIDC) spec, which allows for this type of behavior.

In [ ]:
# Configuration
tenant_id = ''
client_id = ''
redirect_uri = ''

scope = 'openid email profile offline_access https://graph.microsoft.com/.default'
response_type = 'code id_token' #'code id_token token'
response_mode = 'fragment' #'form_post'
state = "A1B2C3D4E5F6"
nonce = '12345'

In [ ]:
complete_auth_url = OAuth2_Flows.hybrid_flow(tenant_id, client_id, redirect_uri, response_type , response_mode, scope, state, nonce)
pyperclip.copy(complete_auth_url) # Copy to clipboard
print(f'Complete URL w/ params - Paste In Browser: \n{complete_auth_url}')

In the response, you should have received the Authorization Code and the ID Token as `code` and `id_token` query string parameters, respectively, within the fragment of the URL. 
Copy the `code` and run below to request an `access_token`.
> [!TIP] 🔥 You can use the [**urlyzer**](https://github.com/ManuelBerrueta/urlyzer) tool to parse the url  for analysis and also to make it easier for you to copy the code 🙂    
> [!NOTE] 📝 You will also need the `client_secret` for this next step.

In [ ]:
auth_code = input('Enter the code from the URL: ')

In [ ]:
client_secret = input('Enter the client secret: ')

Now we can use the `auth_code` and `client_secret` to get the access token and refresh token.

In [ ]:

access_token, refresh_token, id_token = OAuth2_Flows.request_access_token(tenant_id, client_id, redirect_uri, auth_code, client_secret)
print(f'Access Token: {access_token}')
print(f'Refresh Token: {refresh_token}')
print(f'ID Token: {id_token}')

---     

# Refresh Token Request
If the token expires, we can also request a new one using the **`refresh_token`** to make a request to the `/token` endpoint.
> [!NOTE] 📝 The `grant_type` in this case will be `refresh_token`

In [ ]:
refresh_token = input('Enter the refresh token: ')

In [ ]:
client_secret = input('Enter the client secret: ')

In [ ]:
access_token, refresh_token, id_token = OAuth2_Flows.refresh_token(tenant_id, client_id, refresh_token, client_secret)
print(f'Access Token: {access_token}')
print(f'Refresh Token: {refresh_token}')
print(f'ID Token: {id_token}')

## Getting Refresh Token with a different scope
An additional, interesting fact is that you COULD request additional scopes with the refresh token.
> [!NOTE] 📝 The user must have consented to the additional scope to the application at some point or the application must have admin consent.

You can give it a try:

In [ ]:
scope = 'https://vault.azure.net/user_impersonation'

In [ ]:
refresh_token = input('Enter the refresh token: ')

In [ ]:
client_secret = input('Enter the client secret: ')

In [ ]:
tokens = OAuth2_Flows.refresh_token(tenant_id, client_id, refresh_token, client_secret, scope)
access_token = tokens[0]
refresh_token = tokens[1]
id_token = tokens[2]
print(f'Access Token: {access_token}')
print(f'Refresh Token: {refresh_token}')
print(f'ID Token: {id_token}')